In [149]:
import os
import os.path as osp
import pandas as pd
import numpy as np
import torch

In [7]:
def get_edges(x,y):
    src = np.random.randint(0,x,3500)
    dest = np.random.randint(0,y, 3500)
    return [src,dest]

In [151]:
from data import DailyData ,YearlyData

In [201]:
data_list = []
g1 = HeteroData()
g1["node1"].x = torch.tensor(np.round(np.random.rand(1250, 6) * 10), dtype=torch.float)
g1["node2"].x = torch.tensor(np.round(np.random.rand(2000, 6) * 20), dtype=torch.float)
g1["node3"].x = torch.tensor(np.round(np.random.rand(100, 6) * 10), dtype=torch.float)

g1['node2', 'to', 'node1'].edge_index = torch.tensor(get_edges(2000, 1250), dtype=torch.float)
g1['node3', 'to', 'node2'].edge_index = torch.tensor(get_edges(100, 2000), dtype=torch.float)

g1['node2', 'to', 'node1'].edge_attr = torch.tensor(np.round(np.random.rand(3500, 6) * 10), dtype=torch.float)
g1['node3', 'to', 'node2'].edge_attr = torch.tensor(np.round(np.random.rand(3500, 5) * 10), dtype=torch.float)

g1["node2", "to", "node1"].edge_label = torch.rand((3500, 1))
g1["node3", "to", "node2"].edge_label = torch.rand((3500, 1))

node_types, edge_types = g1.metadata()
for node_type in node_types:
    g1[node_type].num_nodes = g1[node_type].x.size(0)

data_list.append(g1)

In [204]:
g1, slices,_ = collate(g1.__class__,data_list = data_list,
                      increment=True,
                      add_batch=True,)
g1

HeteroData(
  node1={
    x=[1250, 6],
    num_nodes=1250,
    batch=[1250],
    ptr=[2]
  },
  node2={
    x=[2000, 6],
    num_nodes=2000,
    batch=[2000],
    ptr=[2]
  },
  node3={
    x=[100, 6],
    num_nodes=100,
    batch=[100],
    ptr=[2]
  },
  (node2, to, node1)={
    edge_index=[2, 350],
    edge_attr=[3500, 6],
    edge_label=[3500, 1]
  },
  (node3, to, node2)={
    edge_index=[2, 350],
    edge_attr=[3500, 5],
    edge_label=[3500, 1]
  }
)

In [206]:
g1["node3"].batch

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [208]:
from torch_geometric.loader import DataLoader
loader = DataLoader(dailydata, batch_size=4)
sampled_hetero_data = next(iter(loader))
sampled_hetero_data["node3"].batch

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

## Create graph structure

In [152]:
# Larget data
root1 = osp.join(os.getcwd(), "dailyroot")
dailydata = DailyData(root1)

In [162]:
dailydata[29]

HeteroData(
  node1={
    x=[125, 6],
    num_nodes=125
  },
  node2={
    x=[200, 6],
    num_nodes=200
  },
  node3={
    x=[10, 6],
    num_nodes=10
  },
  (node2, to, node1)={
    edge_index=[2, 350],
    edge_attr=[350, 6],
    edge_label=[350, 1]
  },
  (node3, to, node2)={
    edge_index=[2, 350],
    edge_attr=[350, 5],
    edge_label=[350, 1]
  }
)

In [165]:
dataset = dailydata.shuffle()

In [184]:
# InMemory data
root2 = osp.join(os.getcwd(), "yearlyroot")
yearlydata = YearlyData(root2)

In [188]:
yearlydata[0]

HeteroData(
  node1={
    x=[1250, 6],
    num_nodes=1250
  },
  node2={
    x=[2000, 6],
    num_nodes=2000
  },
  node3={
    x=[100, 6],
    num_nodes=100
  },
  (node2, to, node1)={
    edge_index=[2, 3500],
    edge_attr=[3500, 6],
    edge_label=[3500, 1]
  },
  (node3, to, node2)={
    edge_index=[2, 3500],
    edge_attr=[3500, 5],
    edge_label=[3500, 1]
  }
)

## Loading - Daily

In [186]:
n = (len(dailydata) + 9) // 10
train_dataset = dailydata[:2*n]
val_dataset = dailydata[2*n:2*n+n]
train_dataset = dailydata[2*n+n:]

In [187]:
train_dataset

DailyData(21)

In [191]:
from torch_geometric.loader import DataLoader
loader = DataLoader(dailydata, batch_size=4)


In [192]:
sampled_hetero_data = next(iter(loader))
sampled_hetero_data

HeteroDataBatch(
  node1={
    x=[500, 6],
    num_nodes=500,
    batch=[500],
    ptr=[5]
  },
  node2={
    x=[800, 6],
    num_nodes=800,
    batch=[800],
    ptr=[5]
  },
  node3={
    x=[40, 6],
    num_nodes=40,
    batch=[40],
    ptr=[5]
  },
  (node2, to, node1)={
    edge_index=[2, 1400],
    edge_attr=[1400, 6],
    edge_label=[1400, 1]
  },
  (node3, to, node2)={
    edge_index=[2, 1400],
    edge_attr=[1400, 5],
    edge_label=[1400, 1]
  }
)

## 1 Yearly graph

In [199]:
from torch_geometric.loader import DataLoader
from torch_geometric.loader import LinkNeighborLoader
loader = LinkNeighborLoader(yearlydata,
                            num_neighbors= {key: [30] * 2 for key in hetero_data.edge_types} ,
                            batch_size=4)


AssertionError: 

In [214]:
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import NeighborLoader

data = Planetoid(os.getcwd(), name='Cora')[0]

loader = LinkNeighborLoader(
    data,
    # Sample 30 neighbors for each node for 2 iterations
    num_neighbors=[30] * 2,
    # Use a batch size of 128 for sampling training nodes
    batch_size=128,
    edge_label_index=data.edge_index,
)

sampled_data = next(iter(loader))
print(sampled_data)

Data(x=[1374, 1433], edge_index=[2, 3102], y=[1374], train_mask=[1374], val_mask=[1374], test_mask=[1374], edge_label_index=[2, 128])


Processing...
Done!


In [225]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [218]:
data.edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

In [248]:
idxs= list(sampled_hetero_data.edge_index_dict.items())
idxs

[(('node2', 'to', 'node1'),
  tensor([[ 99., 143., 156.,  ..., 619., 785., 628.],
          [ 26.,  97., 101.,  ..., 403., 464., 491.]])),
 (('node3', 'to', 'node2'),
  tensor([[  3.,   0.,   7.,  ...,  38.,  30.,  34.],
          [171., 134.,  62.,  ..., 718., 755., 738.]]))]

In [249]:
list(zip(*idxs))

[(('node2', 'to', 'node1'), ('node3', 'to', 'node2')),
 (tensor([[ 99., 143., 156.,  ..., 619., 785., 628.],
          [ 26.,  97., 101.,  ..., 403., 464., 491.]]),
  tensor([[  3.,   0.,   7.,  ...,  38.,  30.,  34.],
          [171., 134.,  62.,  ..., 718., 755., 738.]]))]

In [236]:
edge_label_index = list(sampled_hetero_data.edge_index_dict.items())
edge_label_index

[(('node2', 'to', 'node1'),
  tensor([[ 99., 143., 156.,  ..., 619., 785., 628.],
          [ 26.,  97., 101.,  ..., 403., 464., 491.]])),
 (('node3', 'to', 'node2'),
  tensor([[  3.,   0.,   7.,  ...,  38.,  30.,  34.],
          [171., 134.,  62.,  ..., 718., 755., 738.]]))]

In [239]:
list(sampled_hetero_data.edge_index_dict.values())

[tensor([[ 99., 143., 156.,  ..., 619., 785., 628.],
         [ 26.,  97., 101.,  ..., 403., 464., 491.]]),
 tensor([[  3.,   0.,   7.,  ...,  38.,  30.,  34.],
         [171., 134.,  62.,  ..., 718., 755., 738.]])]

In [231]:
#edge_label_index = {"a",(2,100)}
isinstance(edge_label_index, (list, tuple))

True

In [227]:
d1 = {'x':1,'y':2,'z':3}
l1 = list(d1.items())
l1

[('x', 1), ('y', 2), ('z', 3)]

In [235]:
isinstance(data.edge_index, (list,tuple))

False

In [ ]:
sampled_hetero_data = next(iter(loader))
sampled_hetero_data

### 365 Daily graphs

In [ ]:
dataset = Frankenstein(root='data', pre_transform=T.GCNNorm())

In [51]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.nn import DenseGCNConv as GCNConv, dense_diff_pool

In [58]:
dataset = TUDataset('data', name='PROTEINS')

In [63]:
dataset = dataset.shuffle()

In [65]:
n = (len(dataset) + 9) // 10
test_dataset = dataset[:n]
val_dataset = dataset[n:2 * n]
train_dataset = dataset[2 * n:]

In [74]:
train_loader = DataLoader(train_dataset, batch_size=32)

In [75]:
bat= next(iter(train_loader))
bat    

DataBatch(edge_index=[2, 3538], x=[963, 0], y=[32], batch=[963], ptr=[33])

In [ ]:
def train(model, optimizer, train_loader, device):
    model.train()

    for data in train_loader:
        optimizer.zero_grad()
        data = data.to(device)
        out = model(data.pos, data.batch)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()



val_losses, accs, durations = [], [], []

if torch.cuda.is_available():
        torch.cuda.synchronize()
    t_start = time.perf_counter()

for epoch in range(epochs):
    train_loss = train(model, optimizer, train_loader)
    val_losses.append(eval_loss(model, val_loader))
    accs.append(eval_acc(model, test_loader))
    eval_info = {
        'fold': fold,
        'epoch': epoch,
        'train_loss': train_loss,
        'val_loss': val_losses[-1],
        'test_acc': accs[-1],
    }
    
if torch.cuda.is_available():
    torch.cuda.synchronize()
t_end = time.perf_counter()
durations.append(t_end - t_start)

print(f'Epoch: {epoch:03d}, Test: {test_acc:.4f}, '
              f'Duration: {t_end - t_start:.2f}')

